In [18]:
import pandas as pd
import numpy as np
from fibermagic.IO.NeurophotometricsIO import extract_leds
import os
import pathlib
import copy
from detrend import detrend
# from fibermagic.core.perievents import perievents
from perievents import *



In [19]:
basepath = '/home/james/Massive/PROJECTDATA/NAcC gDA3m + rAdo1.3 FR20-PR/DATA/'

In [20]:
def read_keyfile(keyfile):
    """Opens a csv file containing the metadata about the experiment and returns it as a data frame.
    The columns are expected to be: region, mouse, wave_len, protocol, condition, and include.

    Args:
        keyfile (pathlib.Path): Pathlib path to the keyfile.

    Returns:
        pandas.DataFrame: A dataframe of the experiment metadata.
    """
    abspath = keyfile.absolute()
    keytable = pd.read_csv(abspath)
    return keytable

def read_mouse_log(logfile, id):
    abspath = logfile.absolute()
    logs = pd.read_csv(abspath)
    print(f"Log path is {abspath}")
    logs = pd.read_csv(abspath)
    print(f"Original shape of logs is {logs.shape}")
    logs.columns = ['ComputerTimestamp', 'SystemTimestamp', 'animal.ID', 'Event', 'pi.time', 'pc.time', 'datetimestamp']
    logs = logs[logs['animal.ID']==id]
    print(f"shape of logs is {logs.shape}")
    return logs




def get_photometry(filepath, start_timestamp, logs):
    from fibermagic.IO.NeurophotometricsIO import extract_leds
    # filepath = pathlib.Path(recording['path'] / 'photometry.csv')
    print(f"Photometry path is {filepath}")
    df = pd.read_csv(filepath)
    print(f"Right after opening, the photometry logs are {df.shape}")
    # The column renaming assumes that all columns to the right of "Timestamp" are photometry columns and that the recording regions are letter-number combinations like "X0" or "R1"
    df = df.rename(columns={'R0':'Region0R','G1':'Region1G'})
    df = df.rename(columns={'Timestamp':'SystemTimestamp'})
    df = df[df.SystemTimestamp>=logs.iloc[0]['SystemTimestamp']]
    print(f"Right after filtering, the photometry logs are {df.shape}")
    if 'Flags' in df.columns:  # legacy fix: Flags were renamed to LedState
        df = df.rename(columns={'Flags': 'LedState'})
    # df.astype({'wave_len': 'int32'}).dtypes
    df = extract_leds(df).dropna()
    print(f"After extract LEDs, the photometry logs are {df.shape}")
    return df

# Convert to long format
def convert_to_long(df):
    NPM_RED = 560
    NPM_GREEN = 470
    NPM_ISO = 410
    # dirty hack to come around dropped frames until we find better solution -
    # it makes about 0.16 s difference
    #df = df.iloc[0:]
    df.FrameCounter = np.arange(0, len(df)) // len(df.wave_len.unique())
    df = df.set_index('FrameCounter')
    regions = [column for column in df.columns if 'Region' in column]
    dfs = list()
    for region in regions:
        channel = NPM_GREEN if 'G' in region else NPM_RED
        sdf = pd.DataFrame(data={
            'Region': region,
            'Channel': channel,
            'Timestamp': df.SystemTimestamp[df.wave_len == channel],
            'Signal': df[region][df.wave_len == channel],
            'Reference': df[region][df.wave_len == NPM_ISO]
        }
        )
        dfs.append(sdf)
    dfs = pd.concat(dfs).reset_index().set_index('Region').dropna()
    print(f"After concatenation, the photometry long form is {df.shape}")
    return dfs


def detrend_data(df):
    df = convert_to_long(df)
    df["zdFF"] = detrend(df, "Timestamp", "Signal", "Reference", "Channel", steps=False, method="airPLS", smooth=10, standardize=True)
    print(f"After detrending, the photometry long form is {df.shape}")
    return df

def sync_behavior(logs, detrended):
    import copy
    logs = logs.rename(columns={'SystemTimestamp':'Timestamp'})
    dfsx = copy.deepcopy(detrended)
    dfsx = dfsx.reset_index()
    logsG = pd.merge_asof(logs, dfsx[dfsx.Channel == 470], on="Timestamp", direction = "nearest")
    logsG = logsG[['Region', 'Channel', 'FrameCounter', 'Event', 'Timestamp', 'animal.ID']]
    logsR = pd.merge_asof(logs, dfsx[dfsx.Channel == 560], on="Timestamp", direction = "nearest")
    logsR = logsR[['Region', 'Channel', 'FrameCounter', 'Event', 'Timestamp', 'animal.ID']]
    slogs = pd.concat([logsR, logsG], axis=0)
    slogs = slogs.reset_index(drop=True).set_index(['Region', 'Channel', 'FrameCounter'])
    dfsx = dfsx.reset_index().set_index(['Region', 'Channel', 'FrameCounter'])
    return dfsx, slogs


In [21]:
def get_experiment_dirs(start_path, required_files):
    """Gets the bottom level folders in the starting folder. 
    The assumption going forward will be that each folder contains the files `logs.csv`, `photometry.csv` and `mouse_to_region.csv)

    Args:
        start_path (str): Path to the top-level folder containing the experiment data

    Returns:
        _type_: _description_
    """
    bottom_level_dirs = []
    
    for dirpath, dirnames, filenames in os.walk(start_path):
        # If the current directory has no subdirectories
        if not dirnames:
            # Check if all required files are present in the current directory
            if all(req_file in filenames for req_file in required_files):
                bottom_level_dirs.append(dirpath)
    
    return bottom_level_dirs


def get_experiment_metadata(path):
    experiment = []
    path = pathlib.Path(path)
    keyfile = path / 'region_to_mouse.csv'
    metadata = read_keyfile(keyfile)
    keep = metadata[metadata['include'] == "yes"]
    mice = list(keep['mouse'][keep['include'] == "yes"].unique())
    for mouse_id in mice:
        mouse = {}
        mouse['id'] = mouse_id
        mouse['path'] = path
        mouse['wavelengths'] = list(keep['wave_len'][keep['mouse'] == mouse_id].unique())
        mouse['regions'] = list(keep['region'][keep['mouse'] == mouse_id].unique())
        protocol = list(keep['protocol'][keep['mouse'] == mouse_id].unique())
        if len(protocol) == 1:
            mouse['protocol'] = protocol[0]
        elif len(protocol) > 1:
            raise Exception(f"There are multiple protocols for animal {mouse_id}. That should not be the case. \nPlease check that 'region_to_mouse.csv' is correct")
        else:
            raise Exception(f"There was no protocol found for amimal {mouse_id}. \nPlease check that 'region_to_mouse.csv' is correct")
        condition = list(keep['condition'][keep['mouse'] == mouse_id].unique())
        if len(condition) == 1:
            mouse['condition'] = condition[0]
        elif len(condition) > 1:
            raise Exception(f"There are multiple conditions for animal {mouse_id}. That should not be the case. \nPlease check that 'region_to_mouse.csv' is correct")
        else:
            raise Exception(f"There was no condition found for amimal {mouse_id}. \nPlease check that 'region_to_mouse.csv' is correct. If there was no treatment, enter 'NaN' or None for treatment")
        experiment.append(mouse)
    return experiment




In [22]:
required_files = ['logs.csv', 'photometry.csv', 'region_to_mouse.csv']

experiment_paths = get_experiment_dirs(basepath, required_files)

In [23]:
all_experiments = []

for path in experiment_paths:
    path = pathlib.Path(path)
    experiment = get_experiment_metadata(path)
    all_experiments = all_experiments + experiment

In [24]:
all_experiments

[{'id': 'B1715',
  'path': PosixPath('/home/james/Massive/PROJECTDATA/NAcC gDA3m + rAdo1.3 FR20-PR/DATA/PR11/B1715,B1757-theophylline'),
  'wavelengths': [np.int64(560), np.int64(470)],
  'regions': ['R0', 'G1'],
  'protocol': 'PR11',
  'condition': nan},
 {'id': 'B1757',
  'path': PosixPath('/home/james/Massive/PROJECTDATA/NAcC gDA3m + rAdo1.3 FR20-PR/DATA/PR11/B1715,B1757-saline/repeat'),
  'wavelengths': [np.int64(560), np.int64(470)],
  'regions': ['R2', 'G3'],
  'protocol': 'PR11',
  'condition': 'saline'},
 {'id': 'B1755',
  'path': PosixPath('/home/james/Massive/PROJECTDATA/NAcC gDA3m + rAdo1.3 FR20-PR/DATA/PR11/B1715,B1755'),
  'wavelengths': [np.int64(560), np.int64(470)],
  'regions': ['Region2R', 'Region3G'],
  'protocol': 'PR11',
  'condition': np.float64(nan)},
 {'id': 'B1755',
  'path': PosixPath('/home/james/Massive/PROJECTDATA/NAcC gDA3m + rAdo1.3 FR20-PR/DATA/B1715(PR11),B1755(FR20)'),
  'wavelengths': [np.int64(560), np.int64(470)],
  'regions': ['R2', 'G3'],
  'proto

In [25]:
for experiment in all_experiments:
    mouse_id = experiment['id']
    print(mouse_id, experiment['protocol'])
    print(experiment['path'])
    print(f"Experiment path is{experiment['path']}")
    logfile = experiment['path'] / 'logs.csv'
    experiment['logs'] = read_mouse_log(logfile, mouse_id)
    start_timestamp = experiment['logs'].iloc[0]['SystemTimestamp']
    photometry_file = experiment['path'] / 'photometry.csv'
    experiment['photometry'] = get_photometry(photometry_file, start_timestamp, experiment['logs'])
    experiment['detrended'] = detrend_data(experiment['photometry'])
    # dfsx,slogs = sync_behavior(experiment['logs'], experiment['detrended'])
    # print(dfsx)
    # print(str(experiment['path']))
    # experiment['perievents'] = perievents(dfsx, slogs[slogs.Event=='FD'], window=20, frequency=10)


B1715 PR11
/home/james/Massive/PROJECTDATA/NAcC gDA3m + rAdo1.3 FR20-PR/DATA/PR11/B1715,B1757-theophylline
Experiment path is/home/james/Massive/PROJECTDATA/NAcC gDA3m + rAdo1.3 FR20-PR/DATA/PR11/B1715,B1757-theophylline
Log path is /home/james/Massive/PROJECTDATA/NAcC gDA3m + rAdo1.3 FR20-PR/DATA/PR11/B1715,B1757-theophylline/logs.csv
Original shape of logs is (2752, 7)
shape of logs is (705, 7)
Photometry path is /home/james/Massive/PROJECTDATA/NAcC gDA3m + rAdo1.3 FR20-PR/DATA/PR11/B1715,B1757-theophylline/photometry.csv
Right after opening, the photometry logs are (85006, 8)
Right after filtering, the photometry logs are (82554, 8)
After extract LEDs, the photometry logs are (82554, 9)
After concatenation, the photometry long form is (82554, 8)
After detrending, the photometry long form is (55036, 6)
B1757 PR11
/home/james/Massive/PROJECTDATA/NAcC gDA3m + rAdo1.3 FR20-PR/DATA/PR11/B1715,B1757-saline/repeat
Experiment path is/home/james/Massive/PROJECTDATA/NAcC gDA3m + rAdo1.3 FR20-

In [26]:
for experiment in all_experiments:
    dfsx,slogs = sync_behavior(experiment['logs'], experiment['detrended'])
    experiment['dfsx'] = dfsx
    experiment['slogs'] = slogs
    # try:
    experiment['perievents'] = perievents(dfsx, slogs[slogs.Event=='FD'], window=20, frequency=10)
    # except Exception as e:
    #     experiment['perievents'] = f"ERROR: {str(e)}"
    #     print(e)
    #     continue


/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_event['Timestamp'] = timestamps
/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_event['Timestamp'] = timestamps
/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

"None of ['Event', 'animal.ID', 'Trial'] are in the columns"
Length of values (401) does not match length of index (387)


/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_event['Timestamp'] = timestamps
/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_event['Timestamp'] = timestamps


"None of ['Event', 'animal.ID', 'Trial'] are in the columns"


/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_event['Timestamp'] = timestamps
/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_event['Timestamp'] = timestamps
/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

...

/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_event['Timestamp'] = timestamps
/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_event['Timestamp'] = timestamps
/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

"None of ['Event', 'animal.ID', 'Trial'] are in the columns"


/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_event['Timestamp'] = timestamps
/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_event['Timestamp'] = timestamps
/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

"None of ['Event', 'animal.ID', 'Trial'] are in the columns"


/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_event['Timestamp'] = timestamps
/home/james/Gits/fibermagic-notebooks/perievents.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_event['Timestamp'] = timestamps


In [27]:
all_experiments

[{'id': 'B1715',
  'path': PosixPath('/home/james/Massive/PROJECTDATA/NAcC gDA3m + rAdo1.3 FR20-PR/DATA/PR11/B1715,B1757-theophylline'),
  'wavelengths': [np.int64(560), np.int64(470)],
  'regions': ['R0', 'G1'],
  'protocol': 'PR11',
  'condition': nan,
  'logs':       ComputerTimestamp  SystemTimestamp animal.ID        Event       pi.time  \
  56         6.019064e+07      2554.366560     B1715  video_start -1.000000e+00   
  57         6.019339e+07      2557.112544     B1715           BM  1.725723e+09   
  59         6.020955e+07      2573.264608     B1715           LL  1.725723e+09   
  60         6.021462e+07      2578.340608     B1715           LL  1.725723e+09   
  61         6.022160e+07      2585.320640     B1715           LL  1.725723e+09   
  ...                 ...              ...       ...          ...           ...   
  2689       6.279096e+07      5154.395104     B1715           LL  1.725726e+09   
  2690       6.279478e+07      5158.211136     B1715           LL  1.7257